In [1]:
from getpass import getpass
from huggingface_hub import login

# Enter your token manually (it stays hidden)
HF_TOKEN = getpass("Enter your Hugging Face token: ")

# Login using the entered token
login(token=HF_TOKEN)

print("✅ Environment ready and Hugging Face connected.")


Enter your Hugging Face token: ··········
✅ Environment ready and Hugging Face connected.


In [2]:
!pip install -q transformers peft accelerate bitsandbytes datasets sentencepiece protobuf trl
!pip install -q huggingface_hub

import torch
from huggingface_hub import login
from google.colab import userdata

print("✅ Environment ready and Hugging Face connected.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 8.9 MB/s eta 0:00:00
✅ Environment ready and Hugging Face connected.


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig

BASE_MODEL = "google/gemma-2-2b"

# ✅ Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# ✅ Enable 4-bit quantization (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# ✅ Load base model with quantization
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config
)

# ✅ LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

print("✅ Gemma 2B model loaded with QLoRA successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

✅ Gemma 2B model loaded with QLoRA successfully!


In [4]:
from datasets import load_dataset

DATASET_PATH = "/content/train.jsonl"

# Load dataset from JSONL file
dataset = load_dataset("json", data_files=DATASET_PATH, split="train")

print("✅ Dataset loaded!")
print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

✅ Dataset loaded!
{'input': '<TASK_SUMMARY>\nContext:\n(22)\nHence, from (22), we have\n˙Q(t) ≤( ¯A + 2I)Q(t) + Q(t)( ¯A + 2I)T + ¯Bw ¯BT\nw+\n2α2 ¯Bf ¯BT\nf + 2β2 ¯Bh ¯BT\nh . (23)\nLet Γ(t) = P −Q(t). If condition (20) holds, we use Schur\ncomplements [23] and it follows from (23) that\n˙Γ(t) = −˙Q(t)\n≥−( ¯A + 2I)Q(t) −Q(t)( ¯A + 2I)T −¯Bw ¯BT\nw−\n2α2 ¯Bf ¯BT\nf −2β2 ¯Bh ¯BT\nh\n=−( ¯A+2I)P −P( ¯A+2I)T−¯Bw ¯BT\nw −2α2 ¯Bf ¯BT\nf−\n2β2 ¯Bh ¯BT\nh + (¯A+2I)Γ(t)+Γ(t)(¯A+2I)T\n≥( ¯A + 2I)Γ(t) + Γ(t)( ¯A + 2I)T . (24)\nFrom (24), we obtain Γ(t) ≥e( ¯\nA+2I)tΓ(0)e( ¯\nA+2I)T t. If con-\ndition (20) holds, the matrix ¯A+2I is Hurwitz. It can be thus\nseen that limt→∞Γ(t) ≥0. Then, we have limt→∞Q(t) ≤P,\nwhich implies (21). ■\nFrom the conclusions of Theorem 1 and Theorem 2, we\nhave the following corollary.', 'output': 'The analysis begins by establishing an upper bound for the derivative of a matrix function, $\\dot{Q}(t)$, as presented in inequality (23), which incorporates system matr

In [5]:
print(dataset[0])

{'input': '<TASK_SUMMARY>\nContext:\n(22)\nHence, from (22), we have\n˙Q(t) ≤( ¯A + 2I)Q(t) + Q(t)( ¯A + 2I)T + ¯Bw ¯BT\nw+\n2α2 ¯Bf ¯BT\nf + 2β2 ¯Bh ¯BT\nh . (23)\nLet Γ(t) = P −Q(t). If condition (20) holds, we use Schur\ncomplements [23] and it follows from (23) that\n˙Γ(t) = −˙Q(t)\n≥−( ¯A + 2I)Q(t) −Q(t)( ¯A + 2I)T −¯Bw ¯BT\nw−\n2α2 ¯Bf ¯BT\nf −2β2 ¯Bh ¯BT\nh\n=−( ¯A+2I)P −P( ¯A+2I)T−¯Bw ¯BT\nw −2α2 ¯Bf ¯BT\nf−\n2β2 ¯Bh ¯BT\nh + (¯A+2I)Γ(t)+Γ(t)(¯A+2I)T\n≥( ¯A + 2I)Γ(t) + Γ(t)( ¯A + 2I)T . (24)\nFrom (24), we obtain Γ(t) ≥e( ¯\nA+2I)tΓ(0)e( ¯\nA+2I)T t. If con-\ndition (20) holds, the matrix ¯A+2I is Hurwitz. It can be thus\nseen that limt→∞Γ(t) ≥0. Then, we have limt→∞Q(t) ≤P,\nwhich implies (21). ■\nFrom the conclusions of Theorem 1 and Theorem 2, we\nhave the following corollary.', 'output': 'The analysis begins by establishing an upper bound for the derivative of a matrix function, $\\dot{Q}(t)$, as presented in inequality (23), which incorporates system matrices and various n

In [6]:
def format_example(example):
    return f"""<start_of_turn>
User: {example['input']}
</start_of_turn>
<start_of_turn>
Assistant: {example['output']}
<end_of_turn>"""

# Apply formatting
dataset = dataset.map(lambda x: {"text": format_example(x)})

# Tokenize
def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize, batched=True)

print("✅ Dataset formatted and tokenized!")

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

✅ Dataset formatted and tokenized!


In [7]:
tokenized_dataset

Dataset({
    features: ['input', 'output', 'text', 'input_ids', 'attention_mask'],
    num_rows: 3
})

In [8]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [9]:
!pip install jsonlines

In [10]:
# ---------------- Reinforcement Fine-tune: Gemma 2B (Round 2) ----------------
# Requirements: model & tokenizer already loaded in the notebook (QLoRA model)
#               dataset at /content/data/dataset/train.jsonl
#               HF_TOKEN stored in Colab secrets (userdata)
# This cell reformats dataset with strict instruction templates, does a short
# QLoRA training pass, saves adapters to /content/gemma-finetuned and pushes to HF.

import os
import json
import jsonlines
import random
from datasets import load_dataset, Dataset
import torch

# disable wandb / external logging
os.environ["WANDB_DISABLED"] = "true"

# Paths & params
RAW_DATA_PATH = "/content/train.jsonl"   # your prepared file
REFORMATTED_PATH = "/content/train_reformat.jsonl"
OUTPUT_DIR = "/content/gemma-finetuned"               # overwrite location
NUM_EPOCHS = 2                                        # small reinforcement pass
BATCH_SIZE = 2
GRAD_ACCUM = 4
LR = 2e-4
MAX_LENGTH = 512
SEED = 42
random.seed(SEED)

# ---------- 1) Reformat dataset with strict instruction templates ----------
# We'll read each record and create a clear Instruction/Input/Response style.
# If the input starts with <TASK_SUMMARY> we'll format as a Summary example.
# If <TASK_QUESTIONS>, we'll format as a Questions example.

def reformat_record(rec):
    inp = rec.get("input", "").strip()
    out = rec.get("output", "").strip()
    # Detect task
    if inp.startswith("<TASK_SUMMARY>") or "<TASK_SUMMARY>" in inp:
        # Clean up context extraction: remove token header if present
        # Build strict template (S1 crisp technical)
        context = inp.replace("<TASK_SUMMARY>", "").replace("Context:", "").strip()
        instruction = "Instruction: Summarize the following technical content in a crisp academic tone (≤5 sentences)."
        reform = {
            "instruction": instruction,
            "input": context,
            "response": out.strip()
        }
        return reform
    elif inp.startswith("<TASK_QUESTIONS>") or "<TASK_QUESTIONS>" in inp:
        context = inp.replace("<TASK_QUESTIONS>", "").replace("Context:", "").strip()
        instruction = ("Instruction: Generate EXACTLY 4 academic exam-style questions "
                       "with concise correct answers based on the context below. Use the exact format:\n"
                       "Q1: <question>\nA1: <answer>\nQ2: ...\nA2: ...\nQ3: ...\nA3: ...\nQ4: ...\nA4: ...")
        reform = {
            "instruction": instruction,
            "input": context,
            "response": out.strip()
        }
        return reform
    else:
        # fallback - treat as general
        instruction = "Instruction: Summarize the following text or generate exam-style questions if it is suitable."
        return {"instruction": instruction, "input": inp, "response": out}

# Read original JSONL and write re-formatted JSONL
print("Reformatting dataset...")
count = 0
with jsonlines.open(RAW_DATA_PATH, mode="r") as reader, \
     jsonlines.open(REFORMATTED_PATH, mode="w") as writer:
    for rec in reader:
        # Expect rec to have 'input' and 'output'
        new = reformat_record(rec)
        writer.write(new)
        count += 1

print(f"✅ Reformatted {count} records → {REFORMATTED_PATH}")

# ---------- 2) Load reformatted dataset as HuggingFace Dataset ----------
ds = load_dataset("json", data_files=REFORMATTED_PATH, split="train")
# Optionally shuffle and take a subset for faster run if huge
ds = ds.shuffle(seed=SEED)

# ---------- 3) Build training text in single-field format for causal fine-tuning ----------
def build_text(example):
    # We format as:
    # ### Instruction:
    # <instruction>
    # ### Input:
    # <input>
    # ### Response:
    # <response>
    text = "### Instruction:\n" + example["instruction"].strip() + "\n\n"
    text += "### Input:\n" + example["input"].strip() + "\n\n"
    text += "### Response:\n" + example["response"].strip()
    return {"text": text}

ds = ds.map(lambda x: build_text(x))
# Keep size manageable - take up to ~total 500 examples if you want; we will train on entire ds for reinforcement
# If dataset too large, you can limit, e.g., ds = ds.select(range(400))
print("Dataset size after reformat:", len(ds))

# ---------- 4) Tokenize and prepare for Trainer ----------
from transformers import AutoTokenizer
# assume tokenizer already exists in workspace; otherwise reload:
# tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
def tokenize_func(batch):
    return tokenizer(batch["text"], truncation=True, max_length=MAX_LENGTH, padding="max_length")

tokenized = ds.map(tokenize_func, batched=True, remove_columns=["instruction", "input", "response", "text"])
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])
print("✅ Tokenization complete.")

# ---------- 5) Small QLoRA fine-tune with Trainer ----------
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LR,
    fp16=True,
    logging_steps=20,
    save_strategy="epoch",
    logging_dir=f"{OUTPUT_DIR}/logs",
    remove_unused_columns=False,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Run training (short pass)
print("🚀 Starting reinforcement fine-tuning (short pass)...")
trainer.train()
print("✅ Reinforcement fine-tuning finished.")

# ---------- 6) Save PEFT adapters (overwrite existing) ----------
print("Saving PEFT adapter to", OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)
try:
    # If model is a PEFT model, save_pretrained will save adapter only
    model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print("✅ Saved model+tokenizer to", OUTPUT_DIR)
except Exception as e:
    print("⚠ Could not save via model.save_pretrained():", e)
    # fallback: Hugging Face Trainer save
    trainer.save_model(OUTPUT_DIR)
    print("✅ Saved via trainer.save_model")

# # ---------- 7) Push to Hugging Face (overwrite) ----------
# from huggingface_hub import HfApi
# from google.colab import userdata

# hf_token = userdata.get("HF_TOKEN")
# if not hf_token:
#     print("⚠ HF_TOKEN not found in Colab secrets; skipping push to Hub.")
# else:
#     api = HfApi()
#     # get username
#     who = api.whoami(token=hf_token)
#     username = who.get("name") or who.get("user", {}).get("name")
#     if not username:
#         print("⚠ Could not get username from HF API; skipping push.")
#     else:
#         repo_name = f"{username}/gemma-finetuned"  # overwrite this repo name
#         print("Pushing adapter to hf repo:", repo_name)
#         # create repo if not exists
#         try:
#             api.create_repo(name="gemma-finetuned", token=hf_token, exist_ok=True)
#         except Exception as e:
#             print("Note: create_repo may have failed or repo already exists:", e)
#         # upload folder
#         try:
#             api.upload_folder(
#                 folder_path=OUTPUT_DIR,
#                 path_in_repo=".",
#                 repo_id=repo_name,
#                 token=hf_token,
#                 repo_type="model",
#             )
#             print("✅ Uploaded to Hugging Face repo:", repo_name)
#         except Exception as e:
#             print("⚠ Failed to upload to HF:", e)
#             print("You can manually upload the folder /content/gemma-finetuned to Hugging Face later.")

# print("All done. Your adapter is saved locally and (if HF token present) pushed to your HF account.")
# --------------- End of cell ---------------

Reformatting dataset...
✅ Reformatted 3 records → /content/train_reformat.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset size after reformat: 3


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

✅ Tokenization complete.


/tmp/ipython-input-1419130253.py:135: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 Starting reinforcement fine-tuning (short pass)...


Step,Training Loss


✅ Reinforcement fine-tuning finished.
Saving PEFT adapter to /content/gemma-finetuned
✅ Saved model+tokenizer to /content/gemma-finetuned


In [11]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_PATH = "/content/gemma-finetuned"  # where we saved the adapter

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1300,
    temperature=0.5,
    top_p=0.9,
    repetition_penalty=1.2
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [12]:
prompt = """### Instruction:
Summarize the following technical content in a crisp academic tone (≤5 sentences).

### Input:
Context: Humanity can be viewed as a vast and chaotic chemical reactor, where societies are the turbulent reaction media and individuals are the interacting molecules, each with unique properties. On a psychological level, each mind functions as a sophisticated control system, with our beliefs and desires acting as 'set points' and our senses providing the feedback loop to correct deviations from our goals. Philosophy, in this framework, is the ultimate form of process engineering; it doesn't just ask how to run the reactor more efficiently, but questions the very design of the system, seeking to optimize for abstract 'yields' like justice or happiness. It attempts to define the fundamental differential equations that govern our existence. We are constantly subject to societal 'catalysts'—ideas or technologies that lower the activation energy for change—all while battling the inexorable pull of entropy in both our civilizations and our own consciousness.
### Response:
"""

result = pipe(prompt)[0]['generated_text']
print(result)

### Instruction:
Summarize the following technical content in a crisp academic tone (≤5 sentences).

### Input:
Context: Humanity can be viewed as a vast and chaotic chemical reactor, where societies are the turbulent reaction media and individuals are the interacting molecules, each with unique properties. On a psychological level, each mind functions as a sophisticated control system, with our beliefs and desires acting as 'set points' and our senses providing the feedback loop to correct deviations from our goals. Philosophy, in this framework, is the ultimate form of process engineering; it doesn't just ask how to run the reactor more efficiently, but questions the very design of the system, seeking to optimize for abstract 'yields' like justice or happiness. It attempts to define the fundamental differential equations that govern our existence. We are constantly subject to societal 'catalysts'—ideas or technologies that lower the activation energy for change—all while battling the

In [13]:
prompt = """### Instruction:
Generate EXACTLY 4 academic exam-style questions with concise correct answers based on the context below. Use the format:
Q1: <question>\n
A1: <answer>\n
Q2: <question>\n
A2: <answer>\n
Q3: <question>\n
A3: <answer>\n
Q4: <question>\n
A4: <answer>\n

### Input:
Context: The software development industry is rapidly adopting machine learning for transitioning
modern day software systems towards highly intelligent and self-learning systems. However, the full
potential of machine learning for improving the software engineering life cycle itself is yet to be dis-
covered, i.e., up to what extent machine learning can help reducing the eﬀort/complexity of software
engineering and improving the quality of resulting software systems. To date, no comprehensive study
exists that explores the current state-of-the-art on the adoption of machine learning across software
engineering life cycle stages.
Objective: This article addresses the aforementioned problem and aims to present a state-of-the-art
on the growing number of uses of machine learning in software engineering.
Method: We conduct a systematic mapping study on applications of machine learning to software
engineering following the standard guidelines and principles of empirical software engineering.
Results: This study introduces a machine learning for software engineering (MLSE) taxonomy clas-
sifying the state-of-the-art machine learning techniques according to their applicability to various
software engineering life cycle stages. Overall, 227 articles were rigorously selected and analyzed as
a result of this study.
Conclusion: From the selected articles, we explore a variety of aspects that should be helpful to aca-
demics and practitioners alike in understanding the potential of adopting machine learning techniques
during software engineering projects.

### Response:
"""

result = pipe(prompt)[0]['generated_text']
print(result)

### Instruction:
Generate EXACTLY 4 academic exam-style questions with concise correct answers based on the context below. Use the format:
Q1: <question>

A1: <answer>

Q2: <question>

A2: <answer>

Q3: <question>

A3: <answer>

Q4: <question>

A4: <answer>


### Input:
Context: The software development industry is rapidly adopting machine learning for transitioning
modern day software systems towards highly intelligent and self-learning systems. However, the full
potential of machine learning for improving the software engineering life cycle itself is yet to be dis-
covered, i.e., up to what extent machine learning can help reducing the eﬀort/complexity of software
engineering and improving the quality of resulting software systems. To date, no comprehensive study
exists that explores the current state-of-the-art on the adoption of machine learning across software
engineering life cycle stages.
Objective: This article addresses the aforementioned problem and aims to present a state-of

In [14]:
from huggingface_hub import HfApi
import os
import textwrap

# Variables
model_path = "/content/gemma-finetuned"  # your adapter folder
repo_name = "gemma-study-agent"          # model repo name
hf_username = "ankityadav06"             # your HF username
full_repo_id = f"{hf_username}/{repo_name}"
token = HF_TOKEN  # your Hugging Face token variable

api = HfApi()

# ✅ Create private repo safely
api.create_repo(
    repo_id=full_repo_id,  # FIXED here
    token=token,
    private=True,
    exist_ok=True
)

# ✅ Model Card
model_card = textwrap.dedent(f"""
# Gemma Study Agent – Summary + Exam Q&A from PDFs

This model is a fine-tuned version of **Google Gemma 2B** using **LoRA**.
It converts PDF documents into structured study content:
- ✅ Academic summary
- ✅ Exam-style questions + answers

---

### ✅ Example Usage

\`\`\`python
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model = AutoModelForCausalLM.from_pretrained("{full_repo_id}", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("{full_repo_id}", trust_remote_code=True)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
print(pipe("Summarize control system stability."))
\`\`\`

---

### 📄 License
Apache-2.0
""")

# ✅ Write README
readme_path = os.path.join(model_path, "README.md")
with open(readme_path, "w") as f:
    f.write(model_card)

# ✅ Upload model
api.upload_folder(
    folder_path=model_path,
    path_in_repo=".",
    repo_id=full_repo_id,
    token=token,
    repo_type="model"
)

print(f"✅ Upload complete! View your model at: https://huggingface.co/{full_repo_id}")


<>:61: SyntaxWarning: invalid escape sequence '\`'
<>:53: SyntaxWarning: invalid escape sequence '\`'
<>:61: SyntaxWarning: invalid escape sequence '\`'
<>:53: SyntaxWarning: invalid escape sequence '\`'
/tmp/ipython-input-329512617.py:61: SyntaxWarning: invalid escape sequence '\`'
  token=token,
/tmp/ipython-input-329512617.py:53: SyntaxWarning: invalid escape sequence '\`'
  with open(readme_path, "w") as f:
/usr/local/lib/python3.12/dist-packages/huggingface_hub/hf_api.py:9662: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...checkpoint-1/scheduler.pt: 100%|##########| 1.47kB / 1.47kB            

  ...checkpoint-1/optimizer.pt: 100%|##########| 12.9MB / 12.9MB            

...

  ...finetuned/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...ckpoint-2/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...ckpoint-1/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...point-2/training_args.bin: 100%|##########| 5.78kB / 5.78kB            

✅ Upload complete! View your model at: https://huggingface.co/ankityadav06/gemma-study-agent


In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_PATH = "/content/gemma-finetuned"  # Your fine-tuned model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Inference pipeline
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.5,
    top_p=0.9,
    repetition_penalty=1.1
)

print("✅ Model loaded successfully and ready for inference")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Model loaded successfully and ready for inference


In [16]:
!pip install pypdf

import os
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    """
    Extracts and cleans text from a PDF file.
    """
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"❌ File not found: {pdf_path}")

    text = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    except Exception as e:
        print(f"⚠ Error reading PDF: {e}")

    # Clean text
    text = text.replace("\n", " ").replace("\t", " ").strip()
    return " ".join(text.split())  # Remove extra spaces

print("✅ PDF extraction utility ready.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 12.8 MB/s eta 0:00:00
✅ PDF extraction utility ready.


In [17]:
sample_pdf_path = "/content/Report.pdf"  # change this to your file
extracted_text = extract_text_from_pdf(sample_pdf_path)
print(extracted_text[:10000])  # Preview first 1000 characters


Project Update: Pedestrian Simulation Using Behavioral Heuristics Sanjay Jangir Under the Guidance of Prof. Shankar Prawesh July 2, 2025 1 Introduction This report summarizes progress on my ongoing project to simulate pedestrian move- ment using behavioral heuristics. The core objective is to model how people navigate crowds and obstacles by making intuitive, human-like decisions—such as waiting, step- ping sideways, or following others—rather than relying solely on force-based or optimiza- tion models. The project began with the Vadere simulator for rapid prototyping and later transitioned to the more flexible TaiCrowd framework for deeper customization and experimentation. 2 Project Goals The primary aims of this project are: • Simulate pedestrian movement using intuitive, rule-based heuristics. • Reproduce the decision-making hierarchy described by Seitz et al. (2016): Step-or- Wait → Tangential → Sideways → Follower. • Create realistic environments with walls, bottlenecks, and dyna

In [18]:
import re

def chunk_text(text, max_tokens=800):
    """
    Splits large text into chunks of ~max_tokens words.
    """
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(current_chunk) >= max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

print("✅ Text chunking module ready.")


✅ Text chunking module ready.


In [19]:
chunks = chunk_text(extracted_text)
print(f"Total chunks: {len(chunks)}")
print(chunks[0][:300])  # Preview first chunk


Total chunks: 1
Project Update: Pedestrian Simulation Using Behavioral Heuristics Sanjay Jangir Under the Guidance of Prof. Shankar Prawesh July 2, 2025 1 Introduction This report summarizes progress on my ongoing project to simulate pedestrian move- ment using behavioral heuristics. The core objective is to model 


In [20]:
def generate_summary(text_chunk):
    prompt = f"""### Instruction:
Summarize the following technical content in a crisp academic tone (≤5 sentences).

### Input:
{text_chunk}

### Response:
"""
    result = llm(prompt)[0]['generated_text']
    return result.split("### Response:")[-1].strip()


def summarize_document(chunks):
    print(f"🧠 Generating summary for {len(chunks)} chunks...")
    summaries = []
    for i, chunk in enumerate(chunks):
        print(f"   🔹 Processing chunk {i+1}/{len(chunks)}")
        summary = generate_summary(chunk)
        summaries.append(summary)
    final_summary = "\n".join(summaries)
    return final_summary.strip()


In [21]:
final_summary = summarize_document(chunk_text(extracted_text))
print("✅ Summary Generated:\n")
print(final_summary[:1200])  # Preview first part


🧠 Generating summary for 1 chunks...
   🔹 Processing chunk 1/1
✅ Summary Generated:

Congratulations on your completion of the "Pedestrian Simulation Using Behavioral Heuristics" project! Your summary accurately captures the key points and highlights the importance of simulating pedestrian behavior using intuitive, rule-based heuristics. It's commendable that you utilized various tools and libraries to develop and debug your code effectively. Additionally, incorporating visualiz- ation techniques during the debugging process demonstrates a strong understanding of effective communication. Overall, your summary provides a clear overview of your project's objectives, methodology, and findings. Keep up the good work!


In [22]:
def generate_questions(text_chunk):
    prompt = f"""### Instruction:
Generate EXACTLY 4 academic exam-style questions with concise correct answers based on the context below. Use the format strictly:
Q1: <question>
A1: <answer>
Q2: <question>
A2: <answer>
Q3: <question>
A3: <answer>
Q4: <question>
A4: <answer>

### Input:
{text_chunk}

### Response:
"""
    result = llm(prompt)[0]['generated_text']
    return result.split("### Response:")[-1].strip()


def generate_questions_from_document(chunks):
    print(f"🧠 Generating Q&A for {len(chunks)} chunks...")
    all_qa = []
    for i, chunk in enumerate(chunks):
        print(f"   🔹 Processing chunk {i+1}/{len(chunks)}")
        qa = generate_questions(chunk)
        all_qa.append(qa)
    final_qa = "\n\n".join(all_qa)
    return final_qa


In [23]:
final_qa = generate_questions_from_document(chunk_text(extracted_text))
print("✅ Questions Generated:\n")
print(final_qa[:1200])  # Preview


🧠 Generating Q&A for 1 chunks...
   🔹 Processing chunk 1/1
✅ Questions Generated:

Dear Mr. Jangir, Thank you for submitting your proposal for the research project "Simulating Human Movement Using Behavioral Heuristics." We appreciate your interest in our institution's research opportunities. However, we regret to inform you that your application has been declined due to several reasons: 1. Insufficient Research Experience: Your previous work experience does not demonstrate sufficient expertise in the field of behavioral economics or social psychology. While we recognize that you may be new to these fields, it is important for us to ensure that applicants have relevant background knowledge before undertaking research projects. 2. Lack of Clear Objectives: The proposed study lacks clear objectives and specific goals that would guide the research process. It is crucial for researchers to clearly define what they want to achieve through their studies so that they can develop appropriate m

In [24]:
!apt-get install -y texlive-latex-base texlive-latex-recommended texlive-fonts-recommended texlive-latex-extra


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lato fonts-lmodern fonts-noto-mono
  fonts-texgyre fonts-urw-base35 libapache-pom-java libcommons-logging-java
  libcommons-parent-java libfontbox-java libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1 libruby3.0
  libsynctex2 libteckit0 libtexlua53 libtexluajit2 libwoff1 libzzip-0-13
  lmodern poppler-data preview-latex-style rake ruby ruby-net-telnet
  ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0 rubygems-integration t1utils
  tex-common tex-gyre texlive-base texlive-binaries texlive-pictures
  texlive-plain-generic tipa xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf libavalon-framework-java
  libcommons-logging-java-doc libexcalibur-logkit-java liblog4j1.2-java
  poppler-utils ghostscript fonts-j

In [25]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.4 MB/s eta 0:00:00


In [26]:
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER

def create_pdf(final_summary, final_qa, title="Final Report"):
    # Output file name
    pdf_file = "final_report.pdf"

    # Create document
    doc = SimpleDocTemplate(pdf_file, pagesize=A4,
                            rightMargin=50, leftMargin=50,
                            topMargin=60, bottomMargin=60)

    # Define styles
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle('title_style',
                                 parent=styles['Title'],
                                 alignment=TA_CENTER,
                                 fontSize=22,
                                 textColor=colors.darkblue,
                                 spaceAfter=20)

    heading_style = ParagraphStyle('heading_style',
                                   parent=styles['Heading2'],
                                   fontSize=16,
                                   textColor=colors.darkred,
                                   spaceAfter=12)

    body_style = ParagraphStyle('body_style',
                                parent=styles['Normal'],
                                alignment=TA_JUSTIFY,
                                fontSize=12,
                                leading=18)

    # Build the PDF content
    elements = []

    # Title
    elements.append(Paragraph(title, title_style))
    elements.append(Spacer(1, 20))

    # Section 1: Summary
    elements.append(Paragraph("📘 Final Summary", heading_style))
    elements.append(Paragraph(final_summary.replace("\n", "<br/>"), body_style))
    elements.append(Spacer(1, 20))

    # Section 2: Q&A
    elements.append(PageBreak())
    elements.append(Paragraph("❓ Theoretical Questions & Answers", heading_style))

    # Split Q&A into individual pairs if structured as "Q: ... A: ..."
    qa_blocks = final_qa.split("\n\n")
    for block in qa_blocks:
        elements.append(Paragraph(block.replace("\n", "<br/>"), body_style))
        elements.append(Spacer(1, 10))

    # Save PDF
    doc.build(elements)
    print(f"✅ PDF created successfully: {pdf_file}")
    return pdf_file




# Generate PDF
create_pdf(final_summary, final_qa, title="Engineering PDF Summary + Q&A")


✅ PDF created successfully: final_report.pdf


'final_report.pdf'

In [27]:
!zip -r /content/project_output.zip /content/


  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db (deflated 97%)
  adding: content/.config/.last_update_check.json (deflated 22%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.10.30/ (stored 0%)
  adding: content/.config/logs/2025.10.30/13.35.52.212263.log (deflated 86%)
  adding: content/.config/logs/2025.10.30/13.35.42.751639.log (deflated 58%)
  adding: content/.config/logs/2025.10.30/13.35.10.281184.log (deflated 92%)
  adding: content/.config/logs/2025.10.30/13.36.07.857469.log (deflated 57%)
  adding: content/.config/logs/2025.10.30/13.35.58.184082.log (deflated 58%)
  adding: content/.config/logs/2025.10.30/13.36.08.663879.log (deflated 56%)
  adding: content/.c